In [1]:
import numpy as np
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('/content/train-3.csv')
df.head()

,Accel X,Accel Y,Accel Z,Gyro X,Gyro Y,Gyro Z,Target
0,-0.106800,-0.899333,-0.311333,-4.862667,1.241467,-4.845467,Not Fall
1,-0.168800,-0.918667,-0.323200,-3.316533,1.119867,-5.003333,Not Fall
2,-0.806800,-0.015733,-0.033733,2.998667,1.668800,11.923600,Not Fall
3,-0.863467,-0.027733,-0.022933,4.955867,1.051333,11.309467,Not Fall
4,0.108267,-0.962133,0.058267,-10.379600,9.546400,0.762400,Not Fall


In [4]:
df.dtypes

,0
Accel X,float64
Accel Y,float64
Accel Z,float64
Gyro X,float64
Gyro Y,float64
Gyro Z,float64
Target,object


In [7]:
from sklearn.model_selection import train_test_split

# Handle missing values by replacing with column mean (numerical columns only)
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].apply(lambda col: col.fillna(col.mean()))

# Normalize numeric columns (min-max scaling)
def normalize_column(col):
    return (col - col.min()) / (col.max() - col.min())

df[numeric_columns] = df[numeric_columns].apply(normalize_column)

# Encode target column (e.g., Not Fall -> 0, Fall -> 1)
df['Target'] = df['Target'].apply(lambda x: 0 if x == 'Not Fall' else 1)

# Split the dataset into training and testing sets
X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print("Unique values in y_train:", np.unique(y_train))
print("Data type of y_train:", y_train.dtype)

Unique values in y_train: [0 1]
Data type of y_train: int64


In [9]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (99, 6)
X_test shape: (25, 6)
y_train shape: (99,)
y_test shape: (25,)


In [10]:
# Dense Layer
class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        # Set regularization strength
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    # Forward pass
    def forward(self, inputs, training):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)


        # Gradients on regularization
        # L1 on weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
        # L2 on weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * \
                             self.weights
        # L1 on biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        # L2 on biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * \
                            self.biases

        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

    # Retrieve layer parameters
    def get_parameters(self):
        return self.weights, self.biases

    # Set weights and biases in a layer instance
    def set_parameters(self, weights, biases):
        self.weights = weights
        self.biases = biases


# Dropout
class Layer_Dropout:

    # Init
    def __init__(self, rate):
        # Store rate, we invert it as for example for dropout
        # of 0.1 we need success rate of 0.9
        self.rate = 1 - rate

    # Forward pass
    def forward(self, inputs, training):
        # Save input values
        self.inputs = inputs


        # If not in the training mode - return values
        if not training:
            self.output = inputs.copy()
            return

        # Generate and save scaled mask
        self.binary_mask = np.random.binomial(1, self.rate,
                           size=inputs.shape) / self.rate
        # Apply mask to output values
        self.output = inputs * self.binary_mask

    # Backward pass
    def backward(self, dvalues):
        # Gradient on values
        self.dinputs = dvalues * self.binary_mask


# Input "layer"
class Layer_Input:

    # Forward pass
    def forward(self, inputs, training):
        self.output = inputs

In [11]:
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs, training):
        # Save input values for use during backpropagation
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the gradients
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

    # Predictions method (used for inference)
    def predictions(self, outputs):
        return outputs


# Softmax Activation
class Activation_Softmax:

    def forward(self, inputs, training):
        # Remember input values
        self.inputs = inputs
        # Shift values to prevent overflow
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # Normalize probabilities
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues):
        # Create an uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

    def predictions(self, outputs):
        # Return index of maximum probability
        return np.argmax(outputs, axis=1)

# Sigmoid Activation
class Activation_Sigmoid:

    def forward(self, inputs, training):
        # Remember input values
        self.inputs = inputs
        # Apply sigmoid function
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        # Derivative of sigmoid
        self.dinputs = dvalues * (1 - self.output) * self.output

    def predictions(self, outputs):
        # Return binary classification
        return (outputs > 0.5) * 1

# Tanh Activation
class Activation_Tanh:

    def forward(self, inputs, training):
        # Remember input values
        self.inputs = inputs
        # Apply Tanh activation
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        # Derivative of Tanh
        self.dinputs = dvalues * (1 - self.output ** 2)

    def predictions(self, outputs):
        # Return predictions directly
        return outputs

# Linear Activation
class Activation_Linear:

    def forward(self, inputs, training):
        # Just remember values
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        # Derivative is 1; pass through gradients
        self.dinputs = dvalues.copy()

    def predictions(self, outputs):
        # Return predictions directly
        return outputs

In [12]:
class Optimizer_Adam:

    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        """
        Initialize the Adam optimizer.

        Parameters:
        - learning_rate: Initial learning rate for the optimizer.
        - decay: Learning rate decay factor.
        - epsilon: Small value to prevent division by zero.
        - beta_1: Exponential decay rate for the first moment estimates.
        - beta_2: Exponential decay rate for the second moment estimates.
        """
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def pre_update_params(self):
        """
        Update the learning rate if decay is applied.
        """
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        """
        Update parameters for the given layer using Adam optimization.

        Parameters:
        - layer: A layer object containing weights and biases.
        """
        # If layer does not have moment or cache arrays, initialize them
        if not hasattr(layer, 'weight_momentums'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # Correct momentum for bias correction
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        # Update cache with squared gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights ** 2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases ** 2

        # Correct cache for bias correction
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Update weights and biases
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    def post_update_params(self):
        """
        Increment the iteration count after updating the parameters.
        """
        self.iterations += 1

In [13]:
# Cross-entropy loss
class Loss_CategoricalCrossentropy:

    def forward(self, y_pred, y_true):
        # Clip data to prevent division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # For sparse labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]
        # For one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot encoding
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [14]:
"""class Accuracy_Categorical:

    def calculate(self, y_pred, y_true):
        # Convert predictions to class labels
        predictions = np.argmax(y_pred, axis=1)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        return np.mean(predictions == y_true)"""

class Accuracy_Binary:
    def calculate(self, y_pred, y_true):
        # Convert predictions to binary class labels (ngưỡng 0.5)
        predictions = (y_pred >= 0.5).astype(int)
        return np.mean(predictions == y_true)


In [15]:
class Model:

    def __init__(self):
        self.layers = []
        self.loss = None
        self.optimizer = None
        self.accuracy = None

    def add(self, layer):
        self.layers.append(layer)

    def set(self, *, loss, optimizer, accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuracy = accuracy

    def forward(self, X, training):
        self.layers[0].forward(X, training)
        for i in range(1, len(self.layers)):
            self.layers[i].forward(self.layers[i - 1].output, training)

    def backward(self, y_pred, y_true):
    # Calculate gradient for the loss
        self.loss.backward(y_pred, y_true)

        # Iterate over layers in reverse order
        for i in reversed(range(len(self.layers))):
            # For the last layer, pass the loss gradient
            if i == len(self.layers) - 1:
                self.layers[i].backward(self.loss.dinputs)
            else:
                # Pass the gradient from the next layer
                self.layers[i].backward(self.layers[i + 1].dinputs)

    def summary(self):
        print("Model Summary")
        print("=" * 50)
        total_params = 0
        total_flops = 0

        for layer in self.layers:
            if isinstance(layer, Layer_Dense):
            # Parameters = weights + biases
                params = layer.weights.size + layer.biases.size
                total_params += params
            # FLOPs = (inputs * outputs) + outputs (for biases)
                flops = layer.weights.size + layer.biases.size
                total_flops += flops
                print(f"Dense Layer: {layer.weights.shape[0]} -> {layer.weights.shape[1]}, Params: {params}, FLOPs: {flops}")
            elif isinstance(layer, Activation_ReLU):
                print("Activation Layer: ReLU")
            elif isinstance(layer, Activation_Softmax):
                print("Activation Layer: Softmax")
            elif isinstance(layer, Layer_Dropout):
                print("Dropout Layer")
            else:
                print(f"Layer: {type(layer).__name__}")

        print("=" * 50)
        print(f"Total Parameters: {total_params}")
        print(f"Total FLOPs: {total_flops}")
        print("=" * 50)


    def train(self, X, y, epochs, batch_size, lr_scheduler=None):
        for epoch in range(epochs):
            # Update learning rate if a scheduler is provided
            if lr_scheduler:
                self.optimizer.current_learning_rate = lr_scheduler(epoch)

            # Loop over mini-batches
            for batch_start in range(0, len(X), batch_size):
                batch_end = batch_start + batch_size
                batch_X = X[batch_start:batch_end]
                batch_y = y[batch_start:batch_end]

                # Forward pass
                self.forward(batch_X, training=True)

                # Calculate loss
                loss = np.mean(self.loss.forward(self.layers[-1].output, batch_y))

                # Backward pass
                self.backward(self.layers[-1].output, batch_y)

                # Update weights
                for layer in self.layers:
                    if hasattr(layer, 'weights'):
                        self.optimizer.update_params(layer)

            # Calculate accuracy after each epoch
            accuracy = self.accuracy.calculate(self.layers[-1].output, batch_y)

            # Print metrics for the epoch
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


In [17]:
# Initialize the model
model = Model()

# Add layers to the model
model.add(Layer_Dense(n_inputs=X_train.shape[1], n_neurons=16))
model.add(Activation_ReLU())
model.add(Layer_Dense(n_inputs=16, n_neurons=32))
model.add(Activation_ReLU())
model.add(Layer_Dense(n_inputs=32, n_neurons=1))  # Single output for binary classification
model.add(Activation_Sigmoid())  # Use sigmoid for binary classification

# Set loss, optimizer, and accuracy
model.set(
    loss=Loss_BinaryCrossentropy(),
    optimizer=Optimizer_Adam(learning_rate=0.001),
    accuracy=Accuracy_Binary()
)

# Train the model
model.train(X_train.values, y_train.values.reshape(-1, 1), epochs=50, batch_size=32)


ValueError: operands could not be broadcast together with shapes (3,) (3,2) 

In [ ]:
model.summary()

Model Summary
Dense Layer: 6 -> 16, Params: 112, FLOPs: 112
Activation Layer: ReLU
Dense Layer: 16 -> 32, Params: 544, FLOPs: 544
Activation Layer: ReLU
Dense Layer: 32 -> 2, Params: 66, FLOPs: 66
Activation Layer: Softmax
Total Parameters: 722
Total FLOPs: 722
